In [34]:
import numpy as np

x = np.array([0,0,0,1,1,2,3,0])
# hot_colors = [(0,0,0), (50,0,0), (132,0,0), (170,0,0), (212,0,0), (255,0,0), 
#               (255, 63, 0), (255,103,0), (255,140,0), (255, 170,0), (255,225,0)]
hot_colors = [1,2,3,4,5,6,7,8,9]

for index in range(len(hot_colors)):
    print('Index {} is true at {}'.format(index, np.where(x == index)))
    
heatmap = np.arange(9).reshape((3,3,1))
heat_img = np.zeros(shape=(3,3,3))

print(heatmap)
mask = np.where(heatmap > 5)
print(mask)

heat_img[mask[0], mask[1]] = [99,99,99]

print(heat_img)

Index 0 is true at (array([0, 1, 2, 7], dtype=int64),)
Index 1 is true at (array([3, 4], dtype=int64),)
Index 2 is true at (array([5], dtype=int64),)
Index 3 is true at (array([6], dtype=int64),)
Index 4 is true at (array([], dtype=int64),)
Index 5 is true at (array([], dtype=int64),)
Index 6 is true at (array([], dtype=int64),)
Index 7 is true at (array([], dtype=int64),)
Index 8 is true at (array([], dtype=int64),)
[[[0]
  [1]
  [2]]

 [[3]
  [4]
  [5]]

 [[6]
  [7]
  [8]]]
(array([2, 2, 2], dtype=int64), array([0, 1, 2], dtype=int64), array([0, 0, 0], dtype=int64))
[[[  0.   0.   0.]
  [  0.   0.   0.]
  [  0.   0.   0.]]

 [[  0.   0.   0.]
  [  0.   0.   0.]
  [  0.   0.   0.]]

 [[ 99.  99.  99.]
  [ 99.  99.  99.]
  [ 99.  99.  99.]]]


In [ ]:
frame = '1052'
img = load_image('./project_video-frames/{}.jpg'.format(frame), "RGB")
img = img[360:,550:]

img_rect = img.copy()

boxes = []
visualise = True

def left_far(window):
    dx = -18
    dy = 7
    
    far_windows = []
    
    for i in range(4):
        width = window[1][0] - window[0][0]
        height = window[1][1] - window[0][1]
        
        x1 = window[0][0] + dx
        y1 = window[0][1] + dy
        x2 = x1 + width * 1
        y2 = y1 + height * 0.93
        
        far_window = (int(x1), int(y1)),(int(x2), int(y2))
        far_windows.append(far_window)
        window = far_window
        dx -= -5
        dy -= 1
        
    return far_windows
    
windows = get_boxes(img_rect, 
          x_start_stop=(img.shape[1] - 500, img.shape[1]), 
          y_start_stop=(80, 350), 
          box=(250,200), 
          no_of_boxes = 1, 
          offset_factor=(-0.2, 0))

window = windows[0]
far_windows = left_far(window)

colors = [(255, 0, 0), (0,0,255), (255, 255,0), (255,255,255), (120, 120,120)]

for index, fw in enumerate(far_windows[::-1]):
    color = colors[index % len(colors)]
    cv2.rectangle(img_rect, fw[0], fw[1], color, 4)
    
cv2.rectangle(img_rect, window[0], window[1], (0, 255, 0), 4)

plt.imshow(img_rect)
plt.show()
plt.close('all')


## Sub Sampling of Hog

The issue is that box is a big area (bigger than 64x64) and the hog we have is for the whole image. Even though we can identify the start box number for hog to pick from but now that area won't be 64x64 so it will be many more pixels if we pick the exact box's size and if we don't pick exact box size then we won't have the correct 64x64 area.

In [ ]:
import time

def get_hogs(img):
    features = []
    boxes = get_all_boxes(img)
    
    for box in boxes:
        box_img = img[box[0][1]:box[1][1], box[0][0]:box[1][0]]
        box_img_64 = cv2.resize(box_img, (64,64))    
        features.append(
            get_hog_features(box_img_64[:,:,0], orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True))
    return features
        
def subsample_hog(img, box):
    cell_x = img.shape[1] // pix_per_cell
    
    # in case its the last cell we need to consider the whole block
    # that might even precede this cell
    if cell_x >= feat.shape[1]:
        cell_x = feat.shape[1] - 1
    
    cell_y = img.shape[0] // pix_per_cell
    if cell_y >= feat.shape[0]:
        cell_y = feat.shape[0] - 1
    
    sub_sample = feat[cell_x, cell_y,:,:,:]
    return np.ravel(sub_sample)
    
def get_hogs2(feat_0):
    features = []
    boxes = get_all_boxes(img)

    for box in boxes:
        features.append(subsample_hog(feat_0, box))
    return features
    
filename = './test_images/test1.jpg'
img = load_image(filename, 'RGB')
img_lab = load_image(filename, 'LAB')
print(img.shape)
print(img_lab.shape)

print('Orient', orient, 'Pix / Cell', pix_per_cell, 'Cell / Block', cell_per_block)

feat_0 = get_hog_features(img_lab[:,:,0], orient, pix_per_cell, cell_per_block, vis=False, feature_vec=False)
print(feat_0.shape)

h = get_hogs(img_lab)
h2 = get_hogs2(feat_0)

print(h[0].shape)
print(h[1].shape)

feat_0 = get_hog_features(img_lab[:64,:64,0], orient, pix_per_cell, cell_per_block, vis=False, feature_vec=False)
print(feat_0.shape)